In [ ]:
import logging
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# --- 1. CONFIGURACIÓN DE LA BASE DE DATOS ---
# Usa las mismas credenciales que en tu pipeline
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "EATA"
DB_SCHEMA = "public"
DB_TABLE = "datos_goes"

# Configuración del logging para ver los mensajes
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# --- 2. COMANDOS SQL PARA LA CREACIÓN DE LA ESTRUCTURA ---

# Comando para habilitar la extensión PostGIS si no existe
SQL_CREATE_POSTGIS_EXTENSION = "CREATE EXTENSION IF NOT EXISTS postgis;"

# Comando para crear la tabla principal
SQL_CREATE_TABLE = f"""
CREATE TABLE IF NOT EXISTS {DB_SCHEMA}.{DB_TABLE} (
    id SERIAL PRIMARY KEY,
    producto VARCHAR(50),
    fecha_muestra TIMESTAMP,
    "value" REAL,
    "category" VARCHAR(100),  -- Añadida para la categoría
    "units" VARCHAR(50),      -- Añadida para las unidades
    geom GEOMETRY(Geometry, 4326)
);
"""

# Comando para crear el índice espacial (mejora el rendimiento de las consultas geoespaciales)
SQL_CREATE_GEOM_INDEX = f"""
CREATE INDEX IF NOT EXISTS idx_datos_goes_geom
ON {DB_SCHEMA}.{DB_TABLE} USING GIST (geom);
"""

# Comando para crear el índice de fecha (acelera la eliminación de datos antiguos)
SQL_CREATE_FECHA_INDEX = f"""
CREATE INDEX IF NOT EXISTS idx_datos_goes_fecha
ON {DB_SCHEMA}.{DB_TABLE} (fecha_muestra);
"""

# --- 3. FUNCIÓN DE EJECUCIÓN ---

def configurar_base_de_datos():
    """Se conecta a PostgreSQL y ejecuta los comandos SQL para crear la estructura."""
    
    # URL de conexión
    db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = None
    
    try:
        # Crear la conexión
        engine = create_engine(db_url)
        with engine.connect() as connection:
            logging.info(f"Conectado exitosamente a la base de datos '{DB_NAME}'.")
            
            # Usar una transacción para asegurar que todo se ejecute correctamente
            with connection.begin():
                logging.info("Paso 1/4: Creando extensión PostGIS (si no existe)...")
                connection.execute(text(SQL_CREATE_POSTGIS_EXTENSION))
                logging.info("Extensión PostGIS asegurada.")

                logging.info(f"Paso 2/4: Creando tabla '{DB_TABLE}' (si no existe)...")
                connection.execute(text(SQL_CREATE_TABLE))
                logging.info(f"Tabla '{DB_TABLE}' creada o ya existente.")

                logging.info("Paso 3/4: Creando índice geoespacial (si no existe)...")
                connection.execute(text(SQL_CREATE_GEOM_INDEX))
                logging.info("Índice geoespacial 'idx_datos_goes_geom' creado o ya existente.")
                
                logging.info("Paso 4/4: Creando índice de fecha (si no existe)...")
                connection.execute(text(SQL_CREATE_FECHA_INDEX))
                logging.info("Índice de fecha 'idx_datos_goes_fecha' creado o ya existente.")

            logging.info("\n¡Configuración de la base de datos completada exitosamente!")

    except SQLAlchemyError as e:
        logging.error(f"Ha ocurrido un error durante la configuración de la base de datos: {e}")
    except Exception as e:
        logging.error(f"Un error inesperado ha ocurrido: {e}")
    finally:
        if engine:
            # Cierra todas las conexiones del pool
            engine.dispose()

# --- 4. EJECUCIÓN DEL SCRIPT ---

if __name__ == "__main__":
    configurar_base_de_datos()